In [20]:
!unzip -o "testdaten_edit_06.06.25.zip"

Archive:  testdaten_edit_06.06.25.zip
 extracting: 0d3ca394ec4721bbb7040cc8be8a0e2c1881a50cf08399715eb3305b0769eb4a.json  
 extracting: 3b34ee5ba8285beb05f42aeaa12f39ec10dd6e00643cce3eea50a78086e97d3c.json  
 extracting: 4acf957e62cdf103c91a780b47faed6f08e669a70152f140f83b98072efb462f.json  
 extracting: 4b3445e44d2729ab3ea8723f91e819edd09754dcd5d0e514564110c34322d74d.json  
 extracting: 4fd06b5bff15c1178546f382964793baaf0a0c8af66b92b81dc8b6cd3504aab0.json  
 extracting: 5dcf27da9477a18d0103ef7b61efe7b40cfffd9346eba5d2b777256120ed797c.json  
 extracting: 5dd435a5190b43923ab1ab4173f5cc74e002e9162561df3cc39b037bb0096ada.json  
 extracting: 5fd1ebc847703519f45eededc84f216cbc191cea99390dda1a5fc9b6291e5dcd.json  
 extracting: 06a7dca0f637d59b11620203fd22309cf52ce7cf41363c51e222450a16081a8b.json  
 extracting: 6b848dc7cc0344c674e87e97b0d193f86c8d89bf0751fb4760fd0b7ecc11a53e.json  
 extracting: 6b912d29474e14767f22a621a6cb997dffb4b51781782a0d81d571eec0ab7892.json  
 extracting: 6ed1659f802d30

# Pathling Workshop
## Getting started
### Testdaten Download: https://cloud.uk-erlangen.de/s/qS9BcyYLcpAJWH7

### Relevante Links:
#### https://build.fhir.org/fhirpath.html
#### https://hl7.github.io/fhirpath.js/
#### https://www.basisdatensatz.de/basisdatensatz
#### https://simplifier.net/guide/mii-ig-modul-onkologie-2024-de/MIIIGModulOnkologie?version=current

In [21]:
!apt update
!apt install -y openjdk-17-jdk-headless
!pip install pathling==7.2.0 pandas==2.2.2 matplotlib==3.9.1

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
39 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

# Einführung in die FHIR-Datenextraktion und -analyse mit Pathling

In [22]:
from pathling import PathlingContext, Expression as exp
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, regexp_replace, to_date

spark = (
    SparkSession.builder.config(
        "spark.jars.packages",
        "au.csiro.pathling:library-runtime:7.2.0,",
        # "io.delta:delta-core_2.12:2.4.0,"
        # "org.apache.hadoop:hadoop-aws:3.3.4",
    )
    .config("spark.executor.memory", "8g")
    .config("spark.driver.memory", "10g")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)

spark.sparkContext.setCheckpointDir("/tmp")

pc = PathlingContext.create(
    spark=spark,
    enable_extensions=True,
    enable_delta=True,
    enable_terminology=False,
)

In [27]:
### Load Patient Data
# Wir laden die einzelnen FHIR-Bundles in eine Pathling data source

# !!Vorher müssen die json-Files in den Ordner data/ hier in die Laufzeit hochgeladen werden!!
# TO DO: curl repo and download images here

data = pc.read.bundles(
    "./", ["Patient", "Condition", "Observation", "Procedure"]
)

In [29]:
data

_______________________________________________________________________

# MII Resourcen - Erweiterungsmodul Onkologie
## Implementation Guide
https://simplifier.net/guide/mii-ig-modul-onkologie-2024-de/MIIIGModulOnkologie?version=current

# THEMEN:
## 1. Extract
## 2. Combining Resource Types
### 2.1 Join
### 2.2 Resolve
### 2.3 ReverseResolve
## 3. GROUP EXERCISE: 4 entities
### 3.1 Diagnosis frequency
### 3.2 Gender distribution
### 3.3 Age distribution
### 3.4 Therapy analysis: surgery, radiotherapy
### 3.5 Free Analysis - or time for group work brain storming
________________________________________________________________________

# 1. Extract

## PATIENT -- RESOURCENTYP: PATIENT
https://www.medizininformatik-initiative.de/Kerndatensatz/Modul_Person_Version_2/MIIIGModulPerson-TechnischeImplementierung-FHIR-Profile-PseudonymisiertePatientinPatient.html

In [30]:
### Extract Patient Data

# Wir nutzen `extract`, um nur den Resourcentyp Patient aus der Pathling datasource zu extrahieren und in einen Pyspark Dataframe zu laden
# mithilfe von FHIR Path Ausdrücken können wir in Arrays verschachtelte FHIR Elemente "ausklappen" und als Spalte darstellen
# exp("<FHIR Path Ausdruck>", "<Spaltenname>")

# Dabei extrahieren wir:
# - die einmalige FHIR Resource ID für jede Patientenresource
# - die pseudonymisierte Patienten-ID
# - das Geschlecht
# - das Geburtsdatum
# - das Sterbedatum

patients = data.extract(
    "Patient",
    columns=[
        exp("id", "patient_resource_id"),
        exp(
            "identifier.where(type.coding.where(system='http://terminology.hl7.org/CodeSystem/v2-0203' and code='MR').exists()).value",
            "patid_pseudonym",
        ),
        exp("gender", "gender"),
        exp("birthDate", "birth_date"), # second argument is how we want to name it
        exp("deceasedDateTime", "deceased_datetime"),
    ],
)

# da die extract Funktion relativ resourcenintensiv werden kann in Abhängigkeit der Datenmenge, und da Spark "lazy" agiert, lohnt es sich hier Spark Checkpoint einzusetzen
# "lazy" bedeutet, dass Spark alle Befehle in einer To Do Liste sammelt und erst nach einer "Action" ausführt.
# Außerdem speichert Spark keinen materialisierten Zwischenstand des dataframes, wenn man das nicht ausdrücklich verlangt
# mithilfe des Checkpoints können wir einen solchen Zwischenstand des dataframes zwischenspeichern und verhindern somit, dass Spark dieselben Schritte immer wieder ausführt
# das "count" brauchen wir hier als Action, damit auch der checkpoint sofort ausgeführt wird und nicht aufgrund der "laziness" in die To Do Liste wandert
# mithilfe von spark.sparkContext.setCheckpointDir() können wir den Speicherort der Checkpoints festlegen

patients = patients.checkpoint(eager=True)
patients_count = patients.count()  # enforce checkpoint

print(patients_count)

72


In [31]:
patients.show(72)

+-------------------+----------------+------+----------+-----------------+
|patient_resource_id| patid_pseudonym|gender|birth_date|deceased_datetime|
+-------------------+----------------+------+----------+-----------------+
|   64ffc8f6e2a04bf5|1f8367d0b53064c2|  male|1940-06-22|       2024-04-16|
|   9ba79939ecbcae8f|550d43f7d5cc0b99|  male|1952-03-09|       2019-04-09|
|   114b17715cd8962f|7a258cbfec3c558c|  male|1948-12-06|       2023-11-10|
|   60271b372396f53a|2f4017ec9a0d48c3|female|1954-06-30|       2024-07-25|
|   733b6df2aeaaff06|897acc358f405500|female|1935-08-21|       2019-06-25|
|   ede861350630aad2|65af284f62167c91|  male|1959-06-02|             NULL|
|   150dcba579b4985d|99f5a1dcf9839f0a|  male|2010-12-16|             NULL|
|   3698207ddd269247|033959a00c40ffb4|female|1971-10-30|             NULL|
|   e6573e3c68ea98f2|690fc95f6e6ac7bb|female|1943-09-12|             NULL|
|   847d834b896e3a1d|874de26b0fe8d1ec|  male|1970-05-13|             NULL|
|   ab0f1ed341372001|2b65

In [32]:
# good practice: immer die Gesamtanzahl im Auge behalten
# weil: potentiell explosionsartiges Duplizieren
# z.B. mit count distinct resource ids --> manchmal kann ein zu ungenau spezifiziertes extract der Arrays dazu führen, dass durch das Aufklappen eines Arrays
# mit mehr als einem Element mehrere Zeilen pro Patient (oder Diagnose, etc) erzeugt werden
patients.select("patient_resource_id").distinct().count()

72

In [33]:
# Übung: Wieviele Patient*innen unserer Kohorte sind verstorben und wieviele leben noch bzw. haben keine Sterbeinformation?

## LÖSUNG:
verstorben = patients.filter(col("deceased_datetime").isNotNull()).count()
lebend = patients.filter(col("deceased_datetime").isNull()).count()

print("Verstorben:", verstorben, ", lebend oder unbekannt:", lebend)

Verstorben: 5 , lebend oder unbekannt: 67


## PRIMAERDIAGNOSE -- RESOURCENTYP: CONDITION
https://simplifier.net/guide/mii-ig-modul-onkologie-2024-de/MIIIGModulOnkologie/TechnischeImplementierung/FHIR-Profile/Diagnose/Diagnose-Condition.page.md?version=current

In [ ]:
# wir extrahieren alle Diagnosen - jede Diagnose hat eine eindeutige ID
# wieviele eindeutige Diagnosen haben wir?

conditions = data.extract(
    "Condition",
    columns=[
        exp("id", "condition_resource_id"),
    ],
)
conditions = conditions.checkpoint(eager=True)
conditions_count = conditions.count()  # enforce checkpoint

print(conditions_count)

In [ ]:
# good practice: count distinct ids
conditions.select("condition_resource_id").distinct().count()

In [ ]:
# dazu extrahieren wir den ICD10 Diagnose Code

# "where" in der FHIR Path Expression exp...() ist die Projektion (analog zu WHERE in SQL)
# das "where" im Filter ist die tatsächliche Selektion (analog zu SELECT in SQL)

conditions = data.extract(
    "Condition",
    columns=[
        exp("id", "condition_resource_id"),
        exp(
            "code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').code",  # explizit system angeben, falls es noch weitere Codings mit anderen Systemen gibt
            "icd10_code",
        ),
    ],
    filters=[
        "code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').exists()"  # explizit System nochmal selektieren, um NULL rows zu entfernen, die durch andere Codings ggf. entstehen
    ],
)

conditions = conditions.checkpoint(eager=True)
conditions_count = conditions.count()  # enforce checkpoint

print(conditions_count)

In [ ]:
# Welche Diagnosen kommen in unserem Datensatz vor?

icd10_codes = (
    conditions.select("icd10_code")
    .distinct()
    .orderBy("icd10_code")
    .rdd.flatMap(lambda x: x)
    .collect()
)
icd10_codes

In [ ]:
# Achtung vor explodierenden row counts und Duplikaten!

# die Duplikation entsteht erst wenn ein Element auf condition ebene (höher als code.coding.code Ebene) hinzukommt zum
# framework macht intern einen cross join
# https://pathling.csiro.au/docs/server/operations/extract --> siehe Abschnitt "Notes"
conditions = data.extract(
    "Condition",
    columns=[
        exp("id", "condition_resource_id"),
        exp(
            "code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').code",
            "icd10_code",
        ),  # kardinalität 0...1
        exp(
            "extension('http://hl7.org/fhir/StructureDefinition/condition-assertedDate').valueDateTime",
            "date_diagnosis",
        ),  # kardinalität 1..1
    ],
    filters=[
        "code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').exists()",
    ],
)

conditions = conditions.checkpoint(eager=True)
conditions_count = conditions.count()  # enforce checkpoint

print("Falscher count: ", conditions_count)

In [ ]:
print(
    "distinct id count ist immer noch:",
    conditions.select("condition_resource_id").distinct().count(),
)

In [ ]:
# hier siehst du, dass dupliziert wurde --> jede condition_resource_id kommt jetzt 2x vor
conditions.show()

In [ ]:
# wir brauchen da ein first wo wir sicher sind, dass es nur max. einmal vorkommen kann --> Stichwort Kardinalitäten (0...1 oder 1...1)
conditions = data.extract(
    "Condition",
    columns=[
        exp("id", "condition_resource_id"),
        exp(
            "code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').code.first()",
            "icd10_code",
        ),  # kardinalität 0...1
        exp(
            "extension('http://hl7.org/fhir/StructureDefinition/condition-assertedDate').valueDateTime.first()",
            "date_diagnosis",
        ),  # kardinalität 1..1
        exp("recordedDate", "recorded_date"),
    ],
    filters=[
        "code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').exists()",
    ],
)

conditions = conditions.checkpoint(eager=True)
conditions_count = conditions.count()  # enforce checkpoint

print(conditions_count)

In [ ]:
# hier siehst du jetzt, dass die doppelte Zeile mit condition_resource_id korrekterweise wieder entfernt wurde
conditions.show()

In [ ]:
# Referenz zum Patienten
conditions = data.extract(
    "Condition",
    columns=[
        exp("id", "condition_resource_id"),
        exp(
            "code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').code.first()",
            "icd10_code",
        ),  # kardinalität 0...1
        exp(
            "extension('http://hl7.org/fhir/StructureDefinition/condition-assertedDate').valueDateTime.first()",
            "date_diagnosis",
        ),  # kardinalität 1..1
        exp("subject.reference", "condition_subject_reference"),
    ],
    filters=[
        "code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').exists()"
    ],
)

conditions = conditions.checkpoint(eager=True)
conditions_count = conditions.count()  # enforce checkpoint

print(conditions_count)

In [ ]:
conditions.show()

In [ ]:
# Vergleiche condition_subject_reference aus dem conditions dataframe mit patient_resource_id aus dem patients dataframe -- wo ist der Unterschied?
patients.show()

## JOIN

In [ ]:
# entferne das "Patient/" in den Werten der Spalte "condition_subject_reference", da dieser Präfix in patient_resource_id auch nicht vorkommt
# nach dieser Vorverarbeitung können wir dann joinen: patients["patient_resource_id"] == conditions["condition_subject_reference"]

conditions = conditions.withColumn(
    "condition_subject_reference",
    regexp_replace("condition_subject_reference", "^Patient/", ""),
)
conditions = conditions.checkpoint(eager=True)
conditions.count()  # enforce checkpoint

In [ ]:
conditions.show()

In [ ]:
patients_conditions = (
    patients.alias("p")
    .join(
        conditions.alias("c"),
        patients["patient_resource_id"]
        == conditions["condition_subject_reference"],  # Join condition
        "left",  # Left join
    )
    .select("c.*", "p.*")
)

patients_conditions = patients_conditions.checkpoint(eager=True)
patients_conditions_count = patients_conditions.count()
patients_conditions_count

In [ ]:
patients_conditions.show()

## Resolve

In [ ]:
# deutlich schlankere Alternative zu zwei einzelnen extracts von patients und conditions und join
# resolve löst Referenz auf

conditions_patients_resolve = data.extract(
    "Condition",
    columns=[
        exp("id", "condition_resource_id"),
        exp(
            "code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').code.first()",
            "icd10_code",
        ),  # kardinalität 0...1
        exp(
            "extension('http://hl7.org/fhir/StructureDefinition/condition-assertedDate').valueDateTime.first()",
            "date_diagnosis",
        ),  # kardinalität 1..1
        exp("subject.reference", "condition_subject_reference"),
        exp("subject.resolve().ofType(Patient).id", "patient_resource_id"),
        exp(
            "subject.resolve().ofType(Patient).identifier.where(type.coding.where(system='http://terminology.hl7.org/CodeSystem/v2-0203' and code='MR').exists()).value",
            "patid_pseudonym",
        ),
        exp("subject.resolve().ofType(Patient).birthDate", "birth_date"),
        exp("subject.resolve().ofType(Patient).gender", "gender"),
        exp("subject.resolve().ofType(Patient).deceasedDateTime", "deceased_datetime"),
    ],
    filters=[
        "code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').exists()"
    ],
)
conditions_patients_resolve = conditions_patients_resolve.checkpoint(eager=True)
conditions_patients_resolve_count = conditions_patients_resolve.count()
conditions_patients_resolve_count

In [ ]:
conditions_patients_resolve.show()

## ReverseResolve

In [ ]:
# löst Referenz rückwarts auf

patients_condition_reverse_resolve = data.extract(
    "Patient",
    columns=[
        exp("id", "patient_resource_id"),
        exp(
            "identifier.where(type.coding.where(system='http://terminology.hl7.org/CodeSystem/v2-0203' and code='MR').exists()).value",
            "patid_pseudonym",
        ),
        exp("gender", "gender"),
        exp("birthDate", "birth_date"),
        exp("deceasedDateTime", "deceased_datetime"),
        exp("id", "condition_resource_id"),
        exp(
            "reverseResolve(Condition.subject).code.coding.where(system = 'http://fhir.de/CodeSystem/bfarm/icd-10-gm').code.first()",
            "icd10_code",
        ),  # kardinalität 0...1
        exp(
            "reverseResolve(Condition.subject).extension('http://hl7.org/fhir/StructureDefinition/condition-assertedDate').valueDateTime.first()",
            "date_diagnosis",
        ),  # kardinalität 1..1
        exp(
            "reverseResolve(Condition.subject).subject.reference",
            "condition_subject_reference",
        ),
    ],
)

patients_condition_reverse_resolve = patients_condition_reverse_resolve.checkpoint(
    eager=True
)
patients_condition_reverse_resolve_count = patients_condition_reverse_resolve.count()
patients_condition_reverse_resolve_count

In [ ]:
patients_condition_reverse_resolve.show()

### 3 Lösungswege für dasselbe Ergebnis

In [ ]:
# 1)
print("aus join")
patients_conditions.sort("condition_resource_id").show(2)

# 2)
print("aus resolve:")
conditions_patients_resolve.sort("condition_resource_id").show(2)

# 3
print("aus reverseResolve:")
patients_condition_reverse_resolve.sort("condition_resource_id").show(2)

In [ ]:
### Übung: Altersverteilung der onkologischen Patient:innen

# 1.1 Berechne das Alter bei Diagnose aus dem Geburtsdatum und Diagnosedatum und füge es als neue Spalte in den DataFrame ein
# Hinweis: diese Funktionen sind hilfreich: withColumn und
from pyspark.sql.functions import col, datediff, floor, to_date

patients_conditions = patients_conditions.withColumn(
    "birth_date", to_date(col("birth_date"))
).withColumn(
    "age_at_diagnosis",
    floor(datediff(col("date_diagnosis"), col("birth_date")) / 365.25),
)

patients_conditions.show(truncate=False)

# 1.2 Daten in Pandas überführen

patients_conditions_pd = patients_conditions.toPandas()

# 1.3 Zeichne ein Histogramm der Altersverteilung

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(
    patients_conditions_pd["age_at_diagnosis"].dropna(),
    bins=10,
    edgecolor="black",
    color="blue",
)
plt.xlabel("Alter (Jahre)")
plt.ylabel("Anzahl Patient:innen")
plt.grid(True)
plt.show()

# 3.  GRUPPENARBEIT
### 4 ENTITÄTEN:
##### (Quelle zugegebenermaßen aus Zeitgründen ChatGPT und stichprobenartig überprüft - falls hier etwas gar nicht passt, bitte melden :) )
### 1. Ösophagus C15
#### (Speiseröhrenkrebs)

    Häufigkeit: Relativ selten, aber mit hoher Sterblichkeitsrate – etwa 5.000–6.000 Neuerkrankungen pro Jahr in Deutschland.

    Typen: Zwei Hauptformen – Plattenepithelkarzinom (oberer/mittlerer Ösophagus) und Adenokarzinom (unterer Ösophagus).

    Risikofaktoren: Rauchen, Alkohol, Refluxkrankheit (Barrett-Ösophagus), Adipositas.

    Symptome: Schluckbeschwerden (Dysphagie), Gewichtsverlust, Brustschmerzen.

    Prognose: Sehr schlecht; 5-Jahres-Überlebensrate liegt bei unter 20 %, oft wegen später Diagnose.
### 2. Colon C18
#### (Dickdarmkrebs, ohne Rektum)

    Häufigkeit: Eine der häufigsten Krebsarten – etwa 33.000 Neuerkrankungen pro Jahr in Deutschland.

    Früherkennung: Darmspiegelung (Koloskopie) ab 50 (Männer) bzw. 55 (Frauen) empfohlen.

    Risikofaktoren: Ernährung (viel rotes Fleisch, wenig Ballaststoffe), familiäre Belastung, chronisch entzündliche Darmerkrankungen.

    Symptome: Blut im Stuhl, Veränderungen des Stuhlgangs, unklare Bauchschmerzen.

    Prognose: Bei früher Diagnose gute Heilungschancen – 5-Jahres-Überlebensrate >60 %.
### 3. Pankreas C25
#### (Bauchspeicheldrüsenkrebs)

    Häufigkeit: Etwa 20.000 Neuerkrankungen jährlich in Deutschland, steigende Tendenz.

    Typ: Meist duktales Adenokarzinom des Pankreaskopfes.

    Risikofaktoren: Rauchen, chronische Pankreatitis, Diabetes mellitus, genetische Prädisposition.

    Symptome: Unspezifisch – Gewichtsverlust, Oberbauchschmerzen, Ikterus (Gelbsucht).

    Prognose: Sehr schlecht – 5-Jahres-Überlebensrate unter 10 %, meist späte Diagnose.
### 4. Gliom C71
#### (Hirntumor, maligner, z. B. Glioblastom)

    Häufigkeit: Etwa 6.000–8.000 neue primäre Hirntumoren pro Jahr in Deutschland, davon viele Gliome.

    Typen: Astrozytom, Oligodendrogliom, Glioblastom (besonders aggressiv).

    Risikofaktoren: Meist unklar; wenige bekannte genetische Syndrome (z. B. Li-Fraumeni).

    Symptome: Kopfschmerzen, epileptische Anfälle, neurologische Ausfälle (z. B. Lähmungen, Sprachstörungen).

    Prognose: Stark abhängig vom Grad – Glioblastome (WHO Grad IV) haben eine mittlere Überlebenszeit von ca. 15 Monaten trotz Therapie.
### Aufgabe: Wählt eine Entität aus (jede Entität sollte von mind. einer Gruppe gewählt werden) und führt folgende Analysen durch.
#### 3.1: Wie häufig ist die Diagnose?
#### 3.2: Wie ist die Diagnoseverteilung zwischen den Geschlechtern verteilt?
#### 3.3: Wie sieht die Altersverteilung der Diagnose aus? Visualisiere das Ergebnis, z.B. in einem Histogramm.
#### 3.4: Therapieanalyse: Operation, Strahlentherapie
#### 3.5: Freie Auswertung - was interessiert dich noch innerhalb deiner Kohorte?

In [ ]:
# Lösung 3.1 für C71 Kohorte
patients_conditions_c71 = patients_conditions.filter(
    col("icd10_code").startswith("C71")
)

patients_conditions_c71 = patients_conditions_c71.checkpoint(eager=True)
patients_conditions_c71_count = patients_conditions_c71.count()

print("patients_conditions_c71_count =", patients_conditions_c71_count)

print(
    "distinct condition ids count =",
    patients_conditions_c71.select("condition_resource_id").distinct().count(),
)

In [ ]:
patients_conditions_c71.show()

In [ ]:
# Lösung 3.2 für C71 Kohorte

female = patients_conditions_c71.filter(col("gender") == "female").count()
male = patients_conditions_c71.filter(col("gender") == "male").count()

print("female:", female, ", male:", male)

In [ ]:
# Lösung 3.3 für C71 Kohorte
patients_conditions_c71 = patients_conditions_c71.withColumn(
    "birth_date", to_date(col("birth_date"))
).withColumn(
    "age_at_diagnosis",
    floor(datediff(col("date_diagnosis"), col("birth_date")) / 365.25),
)

patients_conditions_c71.show(truncate=False)

patients_conditions_c71 = patients_conditions_c71.toPandas()

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(
    patients_conditions_pd["age_at_diagnosis"].dropna(),
    bins=10,
    edgecolor="black",
    color="blue",
)
plt.xlabel("Alter (Jahre)")
plt.ylabel("Anzahl Patient:innen")
plt.grid(True)
plt.show()

### 3.4: Therapieanalyse

#### OPERATION -- RESOURCENTYP: PROCEDURE
https://simplifier.net/guide/mii-ig-modul-onkologie-2024-de/MIIIGModulOnkologie/TechnischeImplementierung/FHIR-Profile/Operation/Operation-Procedure.page.md?version=current

In [ ]:
# 3.4 Operationen: untersuche wie häufig deine Kohorte die Therapieform Operation erhält.

# Extrahiere die folgenden Spalten: op resource id, category, op_date, op intention, OPS Code + icd10 code und condition_resource_id der zugrundeliegenden Diagnose
# Welche OP Intentionen findest du in deiner Kohorte, wie oft kommen sie jeweils vor und wofür stehen die Codes? (Stichwort ValueSet im Implementation Guide)
# Extrahiere hierfür die entsprechende Extension (analog zum Diagnosedatum im Einführungsbeispiel).

# Hinweise
# - mithilfe der folgenden Snomed CT category.coding.codes kannst du Operationen filtern:
# 387713003 - Surgical procedure
# 165197003 - Diagnostic assessment
# 394841004 - Other category

# - über Procedure.reasonReference hast du die Verbindung zur zugrundeliegenden Condition Resource = Diagnose.
# - überprüfe dringend jederzeit die distinct op_resource_ids und betrachte den gesamten dataframe, um sicherzustellen, dass du keine duplizierten Zeilen erzeugt hast.

#### STRAHLENTHERAPIE -- RESOURCENTYP: PROCEDURE
https://simplifier.net/guide/mii-ig-modul-onkologie-2024-de/MIIIGModulOnkologie/TechnischeImplementierung/FHIR-Profile/Strahlentherapie/Strahlentherapie-Procedure.page.md?version=current

In [ ]:
# 3.4 Strahlentherapie: untersuche wie häufig deine Kohorte die Therapieform Strahlentherapie erhält.

# Extrahiere die folgenden Spalten: st resource id, category, st_start_date und st_end_date und st_datetime, op intention, OPS Code + icd10 code und condition_resource_id der zugrundeliegenden Diagnose
# Welche OP Intentionen findest du in deiner Kohorte, wie oft kommen sie jeweils vor und wofür stehen die Codes? (Stichwort ValueSet im Implementation Guide)
# Extrahiere hierfür die entsprechende Extension (analog zum Diagnosedatum im Einführungsbeispiel).

# Hinweise
# - mithilfe der folgenden Snomed CT category.coding.codes kannst du Operationen filtern:
# "1287742003" Radiotherapy (procedure)
# "399315003" Radionuclide therapy (procedure)

# - über Procedure.reasonReference hast du die Verbindung zur zugrundeliegenden Condition Resource = Diagnose.
# - überprüfe dringend jederzeit die distinct st_resource_ids und betrachte den gesamten dataframe, um sicherzustellen, dass du keine duplizierten Zeilen erzeugt hast.


# 3.4 Operationen - Lösung:

### 3.5: Freie Auswertung - was interessiert dich noch innerhalb deiner Kohorte?
#### Aufgabe: überlege dir deine eigene Auswertung, erzeuge ggf. einen anschaulichen Plot
#### Inspiration: Geschlecht/Altersverteilung der Therapieformen